In [1]:
def get_library_path()->str:

    cwd = os.getcwd()
    lst = cwd.split(os.sep)[:-1]

    path_lib = f'{os.sep}'

    for elem in lst:
        path_lib = os.path.join(path_lib, elem)

    return path_lib

# imports

import os
import sys

path_to_lib = get_library_path()

sys.path.append(path_to_lib)
import luxgiant_clinical.ThreeCatAnalysis as thr
from luxgiant_clinical.Helpers import categories_recoder

import pandas as pd
import numpy as np

In [2]:
# load data

folder_path = os.path.join(path_to_lib, 'data/source')

df = pd.read_csv(
    os.path.join(folder_path, 'cleaned_file.csv'), 
    low_memory=False
)

In [3]:
# filter data to keep only patients with duration of PD information

off_str = 'OFF: Off is the typical functional state when patien ts have a poor response in spite of taking medications.'

mask_patients= (df['Status']=='Patient')
mask_subtype = (~df['subtype'].isnull())
mask_off     = (df['on_off']==off_str)

df_cases = df[mask_patients & mask_subtype & mask_off].reset_index(drop=True)

del df

df_cases.shape

(1815, 716)

In [5]:
# rename and select first group of variables

variables_dict = {
    "initial_motor_symptom_s___1"   : "Onset with motor symptoms",
    "initial_motor_symptom_s___2"   : "Rest tremor as onset symptom" ,
    "initial_motor_symptom_s___12"  : "Bradykinesia as onset symptom" ,
    "initial_motor_symptom_s___13"  : "Rigidity as onset symptom",
    "initial_motor_symptom_s___3"   : "Postural/action tremor at onset",
    "initial_motor_symptom_s___8"   : "Reduced dexterity at onset",
    "initial_motor_symptom_s___9"   : "Microphagia at onset",
    "initial_motor_symptom_s___11"  : "Reduced arm swing at onset",
    "initial_motor_symptom_s___16"  : "Gait disorder at onset",
    "initial_motor_symptom_s___14"  : "Freezing at onset",
    "initial_motor_symptom_s___15"  : "Falls at onset",
    "initial_non_motor_symptom___1" : "Non Motor Symp at onset",
    "initial_non_motor_symptom___18": "Constipation at onset",
    "initial_non_motor_symptom___8" : "Insomnia as onset symptom",
    "initial_non_motor_symptom___4" : "RBD at onset",
    "initial_non_motor_symptom___4" : "RLS at onset",
    "initial_non_motor_symptom___20": "Urinary urgency as onset symptom",
    "initial_non_motor_symptom___13": "Hyperhidrosis as onset symptom",
    "initial_non_motor_symptom___10": "Anosmia/hyposmia as onset symptom",
    "initial_non_motor_symptom___2" : "Cognitive symptoms at onset",
    "current_motor_symptom_s___2"   : "Current rest tremor",
    "current_motor_symptom_s___3"   : "Current postural/action tremor",
    "current_motor_symptom_s___14"  : "Current bradykinesia",
    "current_motor_symptom_s___4"   : "Current rigidity",
    "current_motor_symptom_s___15"  : "Current Gait disorder",
    "current_motor_symptom_s___16"  : "Current Falls",
    "current_motor_symptom_s___17"  : "Current Freezing",
    "current_motor_symptom_s___18"  : "Current Dyskinesias",
    "current_motor_symptom_s___19"  : "Current Motor fluctuation",
    "current_non_motor_symptom___3" : "Current NMS Psychosis",
    "current_non_motor_symptom___5" : "Current depression",
    "current_non_motor_symptom___6" : "Current RBD",
    "current_non_motor_symptom___7" : "Current RLS",
    "current_non_motor_symptom___9" : "Current anosmia/hyposmia",
    "current_non_motor_symptom___2" : "Current cognitive impairment",
    "current_non_motor_symptom___10": "Current EDSS",
    "current_non_motor_symptom___11": "Current insomnia",
    "current_non_motor_symptom___14": "Current orthostasis",
    "current_non_motor_symptom___16": "Current hyperhidrosis",
    "current_non_motor_symptom___17": "Current seborrhea",
    "current_non_motor_symptom___20": "Current constipation",
    "current_non_motor_symptom___22": "Current urinary incontinence"
}
variables = ['participant_id', 'subtype'] + list(variables_dict.keys())

# statistical measures
stats_meas = {
    'n'     : ['Onset with motor symptoms', 'Rest tremor as onset symptom', 'Bradykinesia as onset symptom', 
               'Rigidity as onset symptom', 'Postural/action tremor at onset', 'Reduced dexterity at onset', 
               'Microphagia at onset', 'Reduced arm swing at onset', 'Gait disorder at onset', 'Freezing at onset', 
               'Falls at onset', 'Non Motor Symp at onset', 'Constipation at onset', 'Insomnia as onset symptom', 
               'RLS at onset', 'Urinary urgency as onset symptom', 'Hyperhidrosis as onset symptom', 
               'Anosmia/hyposmia as onset symptom', 'Cognitive symptoms at onset', 'Current rest tremor', 
               'Current postural/action tremor', 'Current bradykinesia', 'Current rigidity', 'Current Gait disorder', 
               'Current Falls', 'Current Freezing', 'Current Dyskinesias', 'Current Motor fluctuation', 
               'Current NMS Psychosis', 'Current depression', 'Current RBD', 'Current RLS', 
               'Current anosmia/hyposmia', 'Current cognitive impairment', 'Current EDSS', 'Current insomnia', 
               'Current orthostasis', 'Current hyperhidrosis', 'Current seborrhea', 'Current constipation', 
               'Current urinary incontinence']
    }

groups = ['Postural instability and gait difficulty', 'Tremor Dominant', 'Indeterminate']

In [6]:
# keep only variables for analysis

df_cases_1 = df_cases[variables].copy()
df_cases_1 = df_cases_1.rename(columns=variables_dict)

del df_cases

In [7]:
# recode variable values
motor_non_motor = ["Onset with motor symptoms", "Non Motor Symp at onset"]
remainig = [feat for feat in stats_meas['n'] if feat not in motor_non_motor]

df_cases_1 = categories_recoder(df_cases_1, motor_non_motor, {"Checked":0, "Unchecked":1})
df_cases_1 = categories_recoder(df_cases_1, remainig, {"Checked":1, "Unchecked":0})

In [8]:
# count and proportion of categorical variables

df_8 = thr.report_proportion(
    data_df    =df_cases_1,
    variables  =stats_meas['n'],
    groups     =groups,
    grouping_by='subtype',
)

In [9]:
df_cases_2 = df_cases_1[~(df_cases_1['subtype']=='Indeterminate')].copy()
df_81 = thr.bonferroni_proportions(
            data_df      =df_cases_2,
            variables    =stats_meas['n'],
            groups       =['Postural instability and gait difficulty', 'Tremor Dominant'],
            grouping_by  ='subtype',
            correc_factor=3
        )
df_81.columns = ['Variable', 'Adjusted p-value (PIGD vs TD)']

In [10]:
df_8 = thr.final_formatter(
        overall_df=df_8,
        adjusted_df=[df_81],
        groups=groups
    )
df_8

,Variable,Statistical Measure,Postural instability and gait difficulty,Tremor Dominant,Indeterminate,Total,p-value,Adjusted p-value (PIGD vs TD),Available Samples for Analysis
0,Onset with motor symptoms,n (%),851 (99.5),605 (100.0),354 (99.7),1810 (99.7),0.2246,0.4398,1815
1,Rest tremor as onset symptom,n (%),363 (42.5),446 (73.7),217 (61.1),1026 (56.5),p<0.001,p<0.001,1815
2,Bradykinesia as onset symptom,n (%),422 (49.4),172 (28.4),162 (45.6),756 (41.7),p<0.001,p<0.001,1815
3,Rigidity as onset symptom,n (%),184 (21.5),55 (9.1),58 (16.3),297 (16.4),p<0.001,p<0.001,1815
4,Postural/action tremor at onset,n (%),194 (22.7),189 (31.2),95 (26.8),478 (26.3),0.0012,p<0.001,1815
5,Reduced dexterity at onset,n (%),107 (12.5),23 (3.8),21 (5.9),151 (8.3),p<0.001,p<0.001,1815
6,Microphagia at onset,n (%),53 (6.2),30 (5.0),15 (4.2),98 (5.4),0.3234,0.9402,1815
7,Reduced arm swing at onset,n (%),194 (22.7),56 (9.3),49 (13.8),299 (16.5),p<0.001,p<0.001,1815
8,Gait disorder at onset,n (%),60 (7.0),13 (2.1),18 (5.1),91 (5.0),p<0.001,p<0.001,1815
9,Freezing at onset,n (%),21 (2.5),1 (0.2),4 (1.1),26 (1.4),0.0012,0.0012,1815


In [11]:
df_8.to_csv(os.path.join(path_to_lib, 'data/auxiliar/aux_table_8.csv'), index=False)